In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/GemInsights'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptGenerationConfig:
    root_dir: Path
    data_path: Path
    visualization_path: Path
    data_information_file: Path
    prompt_file_name: str
    images_file_name: str
    main_prompt: str

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories, read_text

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        prompt = MAIN_PROMPT_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        self.prompt = read_text(prompt)

        create_directories([self.config.artifacts_root])


    def get_prompt_generation_config(self) -> PromptGenerationConfig:
        config = self.config.prompt_generation
        
        create_directories([config.root_dir])

        prompt_generation_config = PromptGenerationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            visualization_path=config.visualization_path,
            data_information_file=config.data_information_file,
            prompt_file_name=config.prompt_file_name,
            main_prompt=self.prompt,
            images_file_name=config.images_file_name,
        )

        return prompt_generation_config

In [5]:
import pandas as pd
from gemInsights.utils.common import load_json, save_bin, save_txt
import os
import base64
from gemInsights import logger
from vertexai.preview.generative_models import Part

In [6]:
class PromptGeneration:
    def __init__(self, config: PromptGenerationConfig):
        self.config = config

    def generate(self):
        df = pd.read_csv(self.config.data_path)
        info_json = load_json(Path(self.config.data_information_file))
        target_col = info_json.target_col
        # additional_info = info_json.additional_info
        # df_shape = df.shape
        # df_columns = list(df.columns)
        # df_describe = str(df.describe())+"\n"+ str(df.describe(include=["O"]))

        images = []
        image_dir = os.path.join(self.config.visualization_path, target_col)
        image_files = os.listdir(image_dir)
        for image_file in image_files:
            image_path = os.path.join(image_dir, image_file)
            img = open(image_path, "rb").read()
            img_bytes = Part.from_data(base64.b64decode(base64.encodebytes(img)), mime_type="image/jpeg")
            logger.info(f"added the image - {image_file}")
            images.append(img_bytes)

        prompt = f"{self.config.main_prompt}"
            # f"This is the target column of the dataset - '{target_col}'",
            # f"Here are some of the informations related to the dataset - '{additional_info}'",
            # f"The shape of the dataset is {df_shape}",
            # f"The columns in the dataset are {df_columns}",
            # f"Here are some of the general statistics related the dataset - {df_describe}",

        save_bin(data=images, path=Path(os.path.join(self.config.root_dir, self.config.images_file_name)))
        save_txt(data=str(prompt), path=Path(os.path.join(self.config.root_dir, self.config.prompt_file_name)))
        
        

In [7]:
try:
    config = ConfigurationManager()
    prompt_generation_config = config.get_prompt_generation_config()
    prompt_generation = PromptGeneration(config=prompt_generation_config)
    prompt_generation.generate()
except Exception as e:
    raise e

[2023-12-22 01:47:10,100: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-22 01:47:10,102: INFO: common: text file loaded from: main_prompt.txt]
[2023-12-22 01:47:10,103: INFO: common: created directory at: artifacts]
[2023-12-22 01:47:10,103: INFO: common: created directory at: artifacts/prompt_generation]
[2023-12-22 01:47:10,124: INFO: common: json file loaded succesfully from: artifacts/data_ingestion/data_info.json]
[2023-12-22 01:47:10,125: INFO: 876853741: added the image - Scatter_Plots.jpg]
[2023-12-22 01:47:10,126: INFO: 876853741: added the image - Box_Plots.jpg]
[2023-12-22 01:47:10,129: INFO: 876853741: added the image - Bar_Plots.jpg]
[2023-12-22 01:47:10,133: INFO: common: binary file saved at: artifacts/prompt_generation/images.joblib]
[2023-12-22 01:47:10,134: INFO: common: txt file saved at: artifacts/prompt_generation/prompt.txt]
